# Task 2: Data analysis

<font color='red'>**For this Task you will mostly be running the code. Please read carefully, as you don't want to get stuck on a section you you do not need to edit!**</font>

Let us start by restating our working hypothesis:

<font color='blue'>**This can still be changed based on the data**</font>

**Hypothesis 1:** Important events will trigger a peak in twitter activity. 

**Hypothesis 2:** The biggest events will have an impact globaly. 

**Hypothesis 3:** Some peak twitter activity can be matched to events reported by mainstream media.

In the previous task we cleaned and pre-processed twitter daily count data for 2021 to extract peak event dates and number of tweets worldwide, in the USA and in the UK.

We will first compare twitter activity for those key event dates between contries and worldwide but before that we need to setup our coding environment by running the cell below.

<font color='red'>**You DO NOT need to edit anything in the code below!**</font>

In [20]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.linear_model import LinearRegression
import datetime as dt
import numpy as np
from scipy.stats import pearsonr
from sklearn.cluster import KMeans
#sns.set()

Now let us read the cleaned dataset from the previous task. We will call the dataframe **processed**.

In [21]:
processed = pd.read_excel('./input_data/processed_data.xlsx', sheet_name='processed',engine='openpyxl')


We can look at the data again by using the heat map option with pandas.

In [22]:
processed.style.background_gradient(cmap='Blues')

,day,worldwide_events,USA_events,UK_events
0,2021-02-20 00:00:00,2395522.148160,0.000000,0.000000
1,2021-02-28 00:00:00,0.000000,3330580.657522,0.000000
2,2021-03-30 00:00:00,0.000000,2515767.487943,0.000000
3,2021-03-31 00:00:00,5144721.709641,0.000000,5144721.709641
4,2021-04-01 00:00:00,7429007.877885,3765660.476638,7429007.877885
5,2021-04-05 00:00:00,3391570.550858,2386534.454027,3391570.550858
6,2021-04-07 00:00:00,0.000000,3406821.442721,0.000000
7,2021-05-22 00:00:00,0.000000,2310263.188353,0.000000
8,2021-05-26 00:00:00,2733044.131257,4445519.165742,0.000000
9,2021-05-27 00:00:00,0.000000,0.000000,2168840.299500


Looking qualitatively at the data we can already see that there are dates where there is high twitter activity in both the US and the UK and some also correspond to peak activity worldwide. But we also see that sometimes, high twitter activity is present only in one country and not the other.
How can we measure and validate such a relationship between the different activities? We can use a correlation analysis. 

## Correlation analysis

As introduced in <font color='blue'>**Topic correlation**</font>, correlation coefficients will help you identify which countries have a similar or a dissimilar Twitter activity. Not only that but the strength of the coefficients will reflect the strength of this relationship and the sign of the coefficient will correspond to the direction of this relatonship. A positive correlation means that when Twitter activity increase for one country it will also increase in the other country. A negative correlation means the opposite: when Twitter activity increase for one country, it will decrease in the other one.

To run a correlation on the cleaned dataset we just need to apply the **corr** function from pandas to the dataset. There are different correlation methods. We will chose the pearson calculation.

In [23]:
processed.corr(method='pearson')

,worldwide_events,USA_events,UK_events
worldwide_events,1.000000,0.218179,0.692099
USA_events,0.218179,1.000000,-0.188845
UK_events,0.692099,-0.188845,1.000000


What you see above is the correlation matrix between the different attributes. To tell if an attribute is correlated with another, you need to check the coefficient at the intersection of the row and column corresponding to those 2 attributes. For example, the correlation coefficient between worldwide and USA is 0.2. The diagonal of the matrix is always 1 because an attribute is always perfectly correlated to itself and the matrix is symetric because the correlation between an attribute A and an attribute B is the same as between B and A. 

Concerning the interpretation of the coefficients, close to 0 means no correlation, close to 1 is strong positive correlation and close to -1 is strong negative correlation. 

<font color='blue'>**All following interpretation will probably be different when we recieve the real data**</font>

In our case, there is a weak positive correlation between worldwide twitter activity and US (coefficient of 0.2) and a weak negative correlation between the UK and the USA (coefficients of -0.2). In the other hand we see a strong correlation between UK Twitter activity and worldwide (coefficient of 0.7).

how confident are we in our results? Could these relationships we found be due to chance? We have discussed the importance of validating your results in <font color='blue'>**Topic model validation**</font>. One way to validate correlation relationships is to caclculate the p-value. 

## Model validation

To validate our correlation results we will calculate the p-values. In statistics, the p-value is the probability that the result of a measurement is purely due to chance. In our case, a high p-value fort any correlation relationship we find means that the relationship can not be trusted and it can be just a coincidence. If the p-value is small then the correlation relationship is statistically significant. Commonly, we consider any measurement with a p-value smaller than 0.05 as significant.

To calculate the p-value we will define a python function which uses the **pearsonr** function (this uses the same method as for the correlation analysis).

In [24]:
def pearsonr_pval(x,y):
        return pearsonr(x,y)[1]

We can now apply this function to our data.

In [25]:
processed.corr(method=pearsonr_pval)

,worldwide_events,USA_events,UK_events
worldwide_events,1.000000e+00,0.176219,7.547333e-07
USA_events,1.762186e-01,1.000000,2.432009e-01
UK_events,7.547333e-07,0.243201,1.000000e+00


This new matrix contains the corresponding p-values for each attribute pair. Similarly to the correlation coefficients, p-value for the same attributes is always 1 so the diagonal of the matrix is 1 and it is also symetric.

<font color='blue'>**All following interpretation will probably be different when we recieve the real data**</font>

Looking at the results, we see that only the p-value for the relationship between UK events and worldwide events is smaller than 0.05. This means it is the only correlation which is statistically significant and we can be confident in the positive correlation we measured.

The other results are not significants. This means that we cannot confidently say that they are true.

## Comparison between Twitter activity and mainstream media events

Next, we will compare the Twitter peack activity dates with mainstream media event. We already have an excel sheet for the mainstream media data.

We will read the data file in a dataframe called **mainstream_media**.

In [26]:
mainstream_media = pd.read_excel('./input_data/mainstream_media.xlsx', sheet_name='data',engine='openpyxl')

Let us have a look at the data. Remember you can always print out the list of columns or attributes of your dataframe using **mainstream_media.columns**.

In [27]:
mainstream_media

,day,events
0,2021-01-21,Joe Biden Inauguration day
1,2021-02-19,US rejoins Paris climate change accords
2,2021-03-24,MM Ever Given cargo ship blocks the Suez Canal
3,2021-04-21,Derek Chauvin convicted of the murder of Georg...
4,2021-05-03,SpaceX crew arrive home with rare pre-dawn spl...
5,2021-06-24,Colapse of a 12-storey condo building in Florida
6,2021-07-23,Tokyo Olympics opening games ceremony
7,2021-08-02,test
8,2021-08-06,Fall of Afghanistan
9,2021-09-03,Texas bans abortions after 6 weeks


As you can see, the dataframe contains a list of important events which happened worldwide during the year 2021.
This list was compiled from mainstream media news.

## Data format

A first thing you will notice is that the mainstream media data list contains text values. This is what we call a **string** data format. How can we compare strings to numbers? We need to think again about what we are trying to answer. The question we are trying to address is: do the dates with high twitter activity corespond to important events from the mainstream media? And vice versa. At this point we do not care about the scale or count of tweets as we don not have an equivalent mainstream media mesure to compare to. It is only abaout if there is an event on a given day, 'yes' or 'no'. This is what we call a binary solution. We can convert our data into binaries, by replacing all data with twitter counts with the value '1' for yes there is high twitter activity for the corresponding date and everything else will be '0' which means 'no' twitter activity. This is also what we call feature engeneering. We converted our variables into new ones in order to make the comparison between Twitter data and mainstream media data possible.

Before converting the data to binaries, we will first set the column 'day' as the index of our dataframe. This just means that the date is like the key to the data we have. We also do not want to transform the dates.

In [28]:
processed.set_index('day',inplace=True)

Next we just replace any data in the dataframe which are higher than 0 with 1 (this does not impact the index which is the 'day' column).

In [29]:
processed[processed > 0.]=1.

Let us view the data.

In [30]:
processed

,worldwide_events,USA_events,UK_events
day,,,
2021-02-20,1.0,0.0,0.0
2021-02-28,0.0,1.0,0.0
2021-03-30,0.0,1.0,0.0
2021-03-31,1.0,0.0,1.0
2021-04-01,1.0,1.0,1.0
2021-04-05,1.0,1.0,1.0
2021-04-07,0.0,1.0,0.0
2021-05-22,0.0,1.0,0.0
2021-05-26,1.0,1.0,0.0


As you can see, for a given date, the data worldwide, in the USA and in the UK columns have 1 for when there is peak activity in Twitter and 0 if not.

## Combining Twitter data with mainstream media data

We are ready to combine the mainstream media data to the twitter data. This is done by joining the 2 dataframes **'processed'** and **'mainstream_media'**.\
Just before the join, we will also set the column 'day' in the mainstream_media dataframe as an index. This will facilitate the join.

In [31]:
mainstream_media.set_index('day',inplace=True)

We will join the 2 dataframes using the pandas **join** function. The default option will join on the indices of the 2 tables which in this case is 'day' for both. We will use an outer join, which just means that we will keep all the dates from both datasets and not just the overlapping ones. We are also using the sort option so that the final datasets will be nicely structured in ascending dates. We will call the new joined table **media_2021**.

In [32]:
media_2021=processed.join(mainstream_media,how="outer",sort=True)

When you do an outer join. Any data for non-common dates will be filled by Null values (which just means that the value is empty). We do not want to keep any Nulls because numerical calculations usually do not work on missing values. In our case, it makes sens to replace any Null value with 0 as these missing data correspond to non-events. 

The next command replaces the Null values with 0. Remember, you can always check the dataframe by adding commands in the coding cells.

In [33]:
media_2021.fillna(0,inplace=True)

We can review our new joined dataframe and use the background color grading to highlight when there is high twitter activity.

In [34]:
media_2021.style.background_gradient(cmap='Blues')

,worldwide_events,USA_events,UK_events,events
day,,,,
2021-01-21 00:00:00,0.000000,0.000000,0.000000,Joe Biden Inauguration day
2021-02-19 00:00:00,0.000000,0.000000,0.000000,US rejoins Paris climate change accords
2021-02-20 00:00:00,1.000000,0.000000,0.000000,0
2021-02-28 00:00:00,0.000000,1.000000,0.000000,0
2021-03-24 00:00:00,0.000000,0.000000,0.000000,MM Ever Given cargo ship blocks the Suez Canal
2021-03-30 00:00:00,0.000000,1.000000,0.000000,0
2021-03-31 00:00:00,1.000000,0.000000,1.000000,0
2021-04-01 00:00:00,1.000000,1.000000,1.000000,0
2021-04-05 00:00:00,1.000000,1.000000,1.000000,0


<font color='blue'>**All following interpretation will probably be different when we recieve the real data**</font>

As you can see from the table above, qualitatively, or just by checking by eye, we can already recognise what media events correspond to some of twitter activity. <font color='blue'>**[provide few examples when we have the real data]**</font>.

Does that mean that media events as reported by mainstream media drive Twitter activity? To confirm or deny this relationship we will need to run another correlation analysis. This time we will be comparing mainstream media data to each of the Twitter data we have. 

## Correlation analysis and validation with mainstream media data

As mentioned before, we cannot calculate the correlation coefficients based on string type data. We will need to convert the mainstream data to a categorical attribute of '1' for there is a media event and '0' for no media event reported on that day. We will do this using an apply function from pandas which can handle strings.

In [35]:
media_2021['media_num'] = media_2021['events'].apply(lambda x: x if x == 0 else 1)

We can review the new table and see that we now have a new collumn called media_num which containes 1 and 0 values.

In [36]:
media_2021

,worldwide_events,USA_events,UK_events,events,media_num
day,,,,,
2021-01-21,0.0,0.0,0.0,Joe Biden Inauguration day,1
2021-02-19,0.0,0.0,0.0,US rejoins Paris climate change accords,1
2021-02-20,1.0,0.0,0.0,0,0
2021-02-28,0.0,1.0,0.0,0,0
2021-03-24,0.0,0.0,0.0,MM Ever Given cargo ship blocks the Suez Canal,1
2021-03-30,0.0,1.0,0.0,0,0
2021-03-31,1.0,0.0,1.0,0,0
2021-04-01,1.0,1.0,1.0,0,0
2021-04-05,1.0,1.0,1.0,0,0


Time to run the correlation analysis. We will use the same command as above.

In [37]:
media_2021.corr(method='pearson')

,worldwide_events,USA_events,UK_events,media_num
worldwide_events,1.000000,0.127108,0.322190,-0.350438
USA_events,0.127108,1.000000,-0.140286,-0.534522
UK_events,0.322190,-0.140286,1.000000,-0.365148
media_num,-0.350438,-0.534522,-0.365148,1.000000


This time we only care about the relationship between media_num and the Twitter events data.
We find that <font color='blue'>**[provide the insights when we have the real data]**</font>

Again, to confirm the significance of the results we will calculate the p-values calling the pearsonr_pval function we defined above.

In [38]:
media_2021.corr(method=pearsonr_pval)

,worldwide_events,USA_events,UK_events,media_num
worldwide_events,1.000000,0.369208,0.019840,0.010867
USA_events,0.369208,1.000000,0.321227,0.000045
UK_events,0.019840,0.321227,1.000000,0.007774
media_num,0.010867,0.000045,0.007774,1.000000


What this tells me is <font color='blue'>**[provide the insights when we have the real data]**</font>

# Some final remarks

This analysis helps us identifying key event dates and relationships between Twitter activity and mainstream media. The correlations we find, while significant, don't necessarly confirm that that 1 media event is for sure the trigger of increased Twitter activity on a specific day. To really confirm this, one would need to dig deeper in Twitter data by analysing the actual tweets and/or hash tags. That said this is a good start and the identified relationships are still valid within our working assumptions.